<a href="https://colab.research.google.com/github/kevingray92/ATMS-597_Project6/blob/master/GetDataProject6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Import neccessary libraries
from datetime import datetime
import pandas as pd
import numpy as np

!pip install metpy
from metpy.io import parse_metar_to_dataframe
import glob

     |████████████████████████████████| 327kB 2.8MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 194kB 8.8MB/s 


Cannot import USCOUNTIES and USSTATES without Cartopy installed.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Read in the data
##Madison, WI: KMSN
##Green Bay, WI: KGRB
##Oshkosh, WI: KOSH
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2000/64010KOSH*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2001/64010KOSH*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2002/64010KOSH*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2003/64010KOSH*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2004/64010KOSH*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2005/64010KOSH*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2006/64010KOSH*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2007/64010KOSH*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2008/64010KOSH*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2009/64010KOSH*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2010/64010KOSH*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2011/64010KOSH*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2012/64010KOSH*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2013/64010KOSH*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2014/64010KOSH*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2015/64010KOSH*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2016/64010KOSH*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2017/64010KOSH*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2018/64010KOSH*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2019/64010KOSH*.dat'
!wget 'ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2020/64010KOSH*.dat'

--2020-05-03 15:57:19--  ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2000/64010KOSH*.dat
           => ‘.listing’
Resolving ftp.ncdc.noaa.gov (ftp.ncdc.noaa.gov)... 205.167.25.101, 2610:20:8040:2::101
Connecting to ftp.ncdc.noaa.gov (ftp.ncdc.noaa.gov)|205.167.25.101|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/data/asos-fivemin/6401-2000 ... done.
==> PASV ... done.    ==> LIST ... done.

.listing                [  <=>               ] 224.33K   519KB/s    in 0.4s    

2020-05-03 15:57:20 (519 KB/s) - ‘.listing’ saved [229713]

Removed ‘.listing’.
No matches on pattern ‘64010KOSH*.dat’.
--2020-05-03 15:57:20--  ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/6401-2001/64010KOSH*.dat
           => ‘.listing’
Resolving ftp.ncdc.noaa.gov (ftp.ncdc.noaa.gov)... 205.167.25.101, 2610:20:8040:2::101
Connecting to ftp.ncdc.noaa.gov (ftp.ncdc.noaa.gov)|205.167.25.101|:21... connected.
Logging in as a

In [0]:
def parse_metar_file(file, wx_subset=True):
    """
    Parses METAR file from NCDC

    Input:
    --------
    file = Text file downloaded from NCDC

    wx_subset = Flag to determine whether or not to drop non-current weather obs (if True, only returns obs with observed weather)

    Output:
    --------
    df = Pandas dataframe filtered for times where current weather is not 'nan'
    """

    # Read in the file using Pandas
    df = pd.read_csv(file, header=None)

    # Pull the timestamp from the filename
    timestamp = datetime.strptime(file[-10:], '%Y%m.dat')

    # Iterrate over rows to parse METARS
    df_list = []
    for index, row in df.iterrows():
        try:
            df_list.append(parse_metar_to_dataframe(row.values[0][52:], year=timestamp.year, month=timestamp.month))
        except:
            print('Error with METAR: ', row.values[0][52:])
    #
    merged_df = pd.concat(df_list)

    # Drop datasets that do not include current weather
    merged_df = merged_df.dropna(subset=['current_wx1'])
    
    # Change the index to datetime
    merged_df.index = merged_df.date_time

    # Return the merged dataset sorted by datetime
    return merged_df.sort_index()

In [0]:
# Find all files in the data directory
files = sorted(glob.glob('*.dat'))

# Create a list to store the dataframes in
merged_datasets = []

In [0]:
for file in files:
    print(file[-10:])
    try:
        merged_datasets.append(parse_metar_file(file))
    except:
        print("Error with :", file)

200503.dat


/usr/local/lib/python3.6/dist-packages/metpy/calc/basic.py:1011: RuntimeWarning: All-NaN slice encountered
  if np.greater(np.nanmax(np.abs(value)), max_radians):


200504.dat
200505.dat
200506.dat
200507.dat
200508.dat
200509.dat
200510.dat
200511.dat
200512.dat
200601.dat
200602.dat
200603.dat
200604.dat
200605.dat
200606.dat
200607.dat
200608.dat
200609.dat
200610.dat
200611.dat
200612.dat
200701.dat
200702.dat
200703.dat
200704.dat
200705.dat
200706.dat
200707.dat
200708.dat
200709.dat
200710.dat
200711.dat
200712.dat
200801.dat
200802.dat
200803.dat
200804.dat
200805.dat
200806.dat
200807.dat
200808.dat
200809.dat
200810.dat
200811.dat
200812.dat
200901.dat
200902.dat
200903.dat
200904.dat
200905.dat
200906.dat
200907.dat
200908.dat
200909.dat
200910.dat
200911.dat
200912.dat
201001.dat
201002.dat
201003.dat
201004.dat
201005.dat
201006.dat
201007.dat
201008.dat
201009.dat
201010.dat
201011.dat
201012.dat
201101.dat
201102.dat
201103.dat
201104.dat
Error with METAR:   KOSHP201550Z 31006KT 2SM SCT008 OVC014 01/M01 A3000 730 89 -700 310/06 RMK AO2 SFC VIS 8 SNE42 SLP173 P0000 T00111006
201105.dat
201106.dat
201107.dat
201108.dat
201109.dat
2011

In [0]:
del(decoded_metar_master)

NameError: ignored

In [0]:
#clean_df = (merged_datasets)
decoded_metar_master = pd.concat(merged_datasets)

## Save the file to memory
#clean_df.to_csv('decoded_metar_dataset.csv')
# Print out the resulting dataset
decoded_metar_master

,station_id,latitude,longitude,elevation,date_time,wind_direction,wind_speed,current_wx1,current_wx2,current_wx3,low_cloud_type,low_cloud_level,medium_cloud_type,medium_cloud_level,high_cloud_type,high_cloud_level,highest_cloud_type,highest_cloud_level,cloud_coverage,air_temperature,dew_point_temperature,altimeter,present_weather,past_weather,past_weather2,air_pressure_at_sea_level,eastward_wind,northward_wind
date_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2005-03-03 20:35:00,KOSH,43.97,-88.55,248,2005-03-03 20:35:00,220.0,6.0,HZ,NaN,NaN,CLR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,-3.0,-13.0,30.05,5,0,0,NaN,3.856726,4.596267
2005-03-03 20:40:00,KOSH,43.97,-88.55,248,2005-03-03 20:40:00,220.0,5.0,HZ,NaN,NaN,CLR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,-3.0,-14.0,30.05,5,0,0,NaN,3.213938,3.830222
2005-03-04 16:30:00,KOSH,43.97,-88.55,248,2005-03-04 16:30:00,190.0,8.0,-SN,NaN,NaN,OVC,2200,NaN,NaN,NaN,NaN,NaN,NaN,8,-1.0,-6.0,29.79,71,0,0,NaN,1.389185,7.878462
2005-03-04 16:35:00,KOSH,43.97,-88.55,248,2005-03-04 16:35:00,190.0,9.0,-SN,NaN,NaN,OVC,2000,NaN,NaN,NaN,NaN,NaN,NaN,8,-1.0,-6.0,29.79,71,0,0,NaN,1.562834,8.863270
2005-03-04 16:40:00,KOSH,43.97,-88.55,248,2005-03-04 16:40:00,190.0,8.0,-SN,NaN,NaN,OVC,1800,NaN,NaN,NaN,NaN,NaN,NaN,8,-1.0,-6.0,29.79,71,0,0,NaN,1.389185,7.878462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-31 17:20:00,KOSH,43.97,-88.55,248,2020-03-31 17:20:00,20.0,11.0,BR,NaN,NaN,OVC,900,NaN,NaN,NaN,NaN,NaN,NaN,8,1.0,-1.0,30.10,10,0,0,NaN,-3.762222,-10.336619
2020-03-31 17:25:00,KOSH,43.97,-88.55,248,2020-03-31 17:25:00,30.0,11.0,BR,NaN,NaN,OVC,900,NaN,NaN,NaN,NaN,NaN,NaN,8,2.0,0.0,30.10,10,0,0,NaN,-5.500000,-9.526279
2020-03-31 17:30:00,KOSH,43.97,-88.55,248,2020-03-31 17:30:00,30.0,12.0,BR,NaN,NaN,OVC,900,NaN,NaN,NaN,NaN,NaN,NaN,8,1.0,-1.0,30.10,10,0,0,NaN,-6.000000,-10.392305


In [0]:
time = pd.to_datetime(decoded_metar_master['date_time'])
decoded_metar_master['month'] = time.dt.month
decoded_metar_master

,station_id,latitude,longitude,elevation,date_time,wind_direction,wind_speed,current_wx1,current_wx2,current_wx3,low_cloud_type,low_cloud_level,medium_cloud_type,medium_cloud_level,high_cloud_type,high_cloud_level,highest_cloud_type,highest_cloud_level,cloud_coverage,air_temperature,dew_point_temperature,altimeter,present_weather,past_weather,past_weather2,air_pressure_at_sea_level,eastward_wind,northward_wind,month
date_time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2005-03-03 20:35:00,KOSH,43.97,-88.55,248,2005-03-03 20:35:00,220.0,6.0,HZ,NaN,NaN,CLR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,-3.0,-13.0,30.05,5,0,0,NaN,3.856726,4.596267,3
2005-03-03 20:40:00,KOSH,43.97,-88.55,248,2005-03-03 20:40:00,220.0,5.0,HZ,NaN,NaN,CLR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,-3.0,-14.0,30.05,5,0,0,NaN,3.213938,3.830222,3
2005-03-04 16:30:00,KOSH,43.97,-88.55,248,2005-03-04 16:30:00,190.0,8.0,-SN,NaN,NaN,OVC,2200,NaN,NaN,NaN,NaN,NaN,NaN,8,-1.0,-6.0,29.79,71,0,0,NaN,1.389185,7.878462,3
2005-03-04 16:35:00,KOSH,43.97,-88.55,248,2005-03-04 16:35:00,190.0,9.0,-SN,NaN,NaN,OVC,2000,NaN,NaN,NaN,NaN,NaN,NaN,8,-1.0,-6.0,29.79,71,0,0,NaN,1.562834,8.863270,3
2005-03-04 16:40:00,KOSH,43.97,-88.55,248,2005-03-04 16:40:00,190.0,8.0,-SN,NaN,NaN,OVC,1800,NaN,NaN,NaN,NaN,NaN,NaN,8,-1.0,-6.0,29.79,71,0,0,NaN,1.389185,7.878462,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-31 17:20:00,KOSH,43.97,-88.55,248,2020-03-31 17:20:00,20.0,11.0,BR,NaN,NaN,OVC,900,NaN,NaN,NaN,NaN,NaN,NaN,8,1.0,-1.0,30.10,10,0,0,NaN,-3.762222,-10.336619,3
2020-03-31 17:25:00,KOSH,43.97,-88.55,248,2020-03-31 17:25:00,30.0,11.0,BR,NaN,NaN,OVC,900,NaN,NaN,NaN,NaN,NaN,NaN,8,2.0,0.0,30.10,10,0,0,NaN,-5.500000,-9.526279,3
2020-03-31 17:30:00,KOSH,43.97,-88.55,248,2020-03-31 17:30:00,30.0,12.0,BR,NaN,NaN,OVC,900,NaN,NaN,NaN,NaN,NaN,NaN,8,1.0,-1.0,30.10,10,0,0,NaN,-6.000000,-10.392305,3


In [0]:
#decoded_metar_master = decoded_metar_master.dropna(axis=0, subset=['current_wx1','air_temperature'])

## Drop columns we don't plan on using or have too many NaNs
decoded_metar_master = decoded_metar_master.drop(['low_cloud_type',	'low_cloud_level',	'medium_cloud_type',	'medium_cloud_level',	'high_cloud_type',	'high_cloud_level',	'highest_cloud_type',	'highest_cloud_level',	'cloud_coverage', 'air_pressure_at_sea_level'], axis=1)

#decoded_metar_master = decoded_metar_master.dropna(axis=0)

In [0]:
current_wx1 = decoded_metar_master['current_wx1'].to_list()
print(len(current_wx1))
current_wx2 = decoded_metar_master['current_wx2'].to_list()
print(len(current_wx2))
current_wx3 = decoded_metar_master['current_wx3'].to_list()
print(len(current_wx3))

#Check to make sure we have some frozen AND liquid weather reported
print(current_wx1[0:50])

256254
256254
256254
['HZ', 'HZ', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', '-SN', 'SN', 'SN', '-SN', '-SN', '-SN', 'SN', 'SN', 'SN', 'SN', '-SN', '-SN', '-SN', '-SN', 'BR', 'BR', 'BR', 'BR', 'BR', 'BR', 'HZ', 'HZ', 'BR', 'HZ', 'HZ', 'HZ', 'HZ', 'HZ', 'HZ', 'HZ']


In [0]:
# List all of the current weather identifiers to identify non-precipitation identifiers to be removed
print(decoded_metar_master.current_wx1.unique())

['HZ' '-SN' 'SN' 'BR' 'FZFG' '-RA' '+SN' '-FZRA' 'TS' '-TSRA' '+RA' 'VCTS'
 'FG' 'RA' 'TSRA' '+TSRA' 'FZRA' 'BL' '-TS']


In [0]:
# Create non-precip variables list to replace with NaNs
non_precip = ['HZ', 'BR', 'FG', '-FZDZ', 'BL', 'TS', 'VCTS', 'MIFG', 'FZFG', '-SNDZ', 'VCFG', '+TS', '-TS', 'FZDZ', '-TSDZ']

In [0]:
# Replace non-precip variables with NaNs
decoded_metar_master.replace(non_precip, np.nan, inplace=True)
### Then drop current_wx2 and current_wx3 because they hold a lot of NaNs
decoded_metar_master = decoded_metar_master.drop(['current_wx2',	'current_wx3'], axis=1)
### Then drop all other rows with NaNs
decoded_metar_master = decoded_metar_master.dropna(axis=0)
### Check how many rows we are left with after cleaning up the dataframe
current_wx1 = decoded_metar_master['current_wx1'].to_list()
print(len(current_wx1))

105656


In [0]:
### List of frozen and liquid precip options
fzn_list = ['-SNRA','SNRA','+SNRA','-SN','SN','+SN', '-TSSN', 'TSSN', '+TSSN']
liq_list = ['-RA','RA','+RA','-TS','TS','+TS','-TSRA','TSRA','+TSRA','-FZRA','FZRA','+FZRA']

In [0]:
### Set up a list to hold binary values (1 for frozen, 0 for liquid)
binary_list = []

### Loop over current_wx observations and determin if any are frozen precip. If so, set binary variable to 1. If not, set binary variable to 0.
for i in range(len(current_wx1)):
    if (current_wx1[i] in fzn_list) or (current_wx2[i] in fzn_list) or (current_wx3[i] in fzn_list):
        binary_list.append(1)
    else:
        binary_list.append(0)

### Fill a new column called 'fzn_or_liq' with the binary values.
decoded_metar_master['fzn_or_liq'] = binary_list

In [0]:
# Double check remaining parameters are correct and all precipitation
decoded_metar_master.current_wx1.unique()

array(['-SN', 'SN', '-RA', '+SN', '-FZRA', '-TSRA', '+RA', 'RA', 'TSRA',
       '+TSRA', 'FZRA'], dtype=object)

In [0]:
decoded_metar_master

,station_id,latitude,longitude,elevation,date_time,wind_direction,wind_speed,current_wx1,air_temperature,dew_point_temperature,altimeter,present_weather,past_weather,past_weather2,eastward_wind,northward_wind,month,fzn_or_liq
date_time,,,,,,,,,,,,,,,,,,
2005-03-04 16:30:00,KOSH,43.97,-88.55,248,2005-03-04 16:30:00,190.0,8.0,-SN,-1.0,-6.0,29.79,71,0,0,1.389185,7.878462,3,1
2005-03-04 16:35:00,KOSH,43.97,-88.55,248,2005-03-04 16:35:00,190.0,9.0,-SN,-1.0,-6.0,29.79,71,0,0,1.562834,8.863270,3,1
2005-03-04 16:40:00,KOSH,43.97,-88.55,248,2005-03-04 16:40:00,190.0,8.0,-SN,-1.0,-6.0,29.79,71,0,0,1.389185,7.878462,3,1
2005-03-04 16:45:00,KOSH,43.97,-88.55,248,2005-03-04 16:45:00,200.0,9.0,-SN,-1.0,-6.0,29.79,71,0,0,3.078181,8.457234,3,1
2005-03-04 16:50:00,KOSH,43.97,-88.55,248,2005-03-04 16:50:00,190.0,8.0,-SN,-1.0,-6.0,29.79,71,0,0,1.389185,7.878462,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-31 13:55:00,KOSH,43.97,-88.55,248,2020-03-31 13:55:00,30.0,10.0,-SN,1.0,-1.0,30.11,71,10,0,-5.000000,-8.660254,3,1
2020-03-31 14:00:00,KOSH,43.97,-88.55,248,2020-03-31 14:00:00,20.0,11.0,-SN,1.0,-1.0,30.11,71,10,0,-3.762222,-10.336619,3,1
2020-03-31 14:05:00,KOSH,43.97,-88.55,248,2020-03-31 14:05:00,20.0,12.0,-SN,1.0,-1.0,30.11,71,10,0,-4.104242,-11.276311,3,1


In [0]:
# Save the final data to a csv file
decoded_metar_master.to_csv('/content/drive/My Drive/decoded_metar_FINAL_Proj6_KOSH.csv')